In [1]:
# Input

from collections import OrderedDict, defaultdict
import threading

class CacheLevel:
    def __init__(self, size, eviction_policy):
        self.size = size
        self.eviction_policy = eviction_policy
        self.cache = OrderedDict() if eviction_policy == 'LRU' else {}
        if eviction_policy == 'LFU':
            self.frequency = defaultdict(int)  # To track frequency of access

    def get(self, key):
        if self.eviction_policy == 'LRU':
            if key in self.cache:
                self.cache.move_to_end(key)  # Move to the end to mark as recently used
                return self.cache[key]
        elif self.eviction_policy == 'LFU':
            if key in self.cache:
                self.frequency[key] += 1  # Increase frequency count
                return self.cache[key]
        return None

    def put(self, key, value):
        if key in self.cache:
            if self.eviction_policy == 'LRU':
                self.cache.move_to_end(key)
            elif self.eviction_policy == 'LFU':
                self.frequency[key] += 1
            self.cache[key] = value
        else:
            if len(self.cache) >= self.size:
                self.evict()
            if self.eviction_policy == 'LFU':
                self.frequency[key] = 1
            self.cache[key] = value

    def evict(self):
        if self.eviction_policy == 'LRU':
            self.cache.popitem(last=False)  # Remove least recently used
        elif self.eviction_policy == 'LFU':
            lfu_key = min(self.frequency, key=self.frequency.get)
            del self.cache[lfu_key]
            del self.frequency[lfu_key]

    def display(self):
        if self.eviction_policy == 'LRU':
            return list(self.cache.items())
        elif self.eviction_policy == 'LFU':
            return list(self.cache.items())

class MultilevelCacheSystem:
    def __init__(self):
        self.cache_levels = []
        self.lock = threading.Lock()  # For thread safety

    def addCacheLevel(self, size, eviction_policy):
        with self.lock:
            self.cache_levels.append(CacheLevel(size, eviction_policy))

    def removeCacheLevel(self, level):
        with self.lock:
            if 0 <= level < len(self.cache_levels):
                self.cache_levels.pop(level)

    def get(self, key):
        with self.lock:
            for i, cache in enumerate(self.cache_levels):
                result = cache.get(key)
                if result is not None:
                    self.promote(i, key, result)
                    return result
        return None

    def promote(self, current_level, key, value):
        for i in range(current_level, 0, -1):
            self.cache_levels[i - 1].put(key, value)

    def put(self, key, value):
        with self.lock:
            if len(self.cache_levels) > 0:
                self.cache_levels[0].put(key, value)

    def displayCache(self):
        with self.lock:
            for i, cache in enumerate(self.cache_levels):
                print(f"L{i + 1} Cache: {cache.display()}")
